# Cuaderno de creación de lista negra de usuarios

El código definido a continuación tiene como objetivo analizar las publicaciones de la carpeta [`datos`](./datos), y crear una lista de usuarios que no deberían ser considerados en el análisis de datos.

Se consideraran para esta lista usuarios que sean autores de: 

* Publicaciones hechas para fomentar el cuidado de la salud mental
    * Si bien se pueden encontrar palabras clave que indiquen tendencias suicidas, no necesariamente reflejan problemas mentales en quien las publica
* Publicaciones repetidas (spam, bots, ofertas de trabajo, etc.)
* Publicaciones de páginas de noticias

Además, es posible excluir tweets que se consideren irrelevantes para el análisis, como por ejemplo:

* Aquellos que usen una de las palabras claves del conjunto de datos, pero que no tengan relación con el tema
    * Ejemplo: "Mi hermano suele colgarme cuando le llamo"

Para ejecutar las siguientes instrucciones, se empleará el lenguaje de R.

Será de utilidad en el análisis, además, los paquetes `tm` y `SnowballC`, que permiten realizar análisis sobre texto y lematización.

In [1]:
# Se cargan las librerías
library('tm')
library('dplyr')
library('SnowballC')

Warning message:
"package 'tm' was built under R version 4.2.3"
Loading required package: NLP



Warning message:
"package 'dplyr' was built under R version 4.2.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'SnowballC' was built under R version 4.2.3"


Primero que nada, definimos funciones que nos permitan manipular qué archivos queremos analizar.

Estas funciones permiten procesar de manera genérica los archivos de la carpeta [`datos`](./datos), de tal forma que se contemplen únicamente los registros de usuarios con más de N publicaciones.

Otras consideraciones sobre los datos serán más específicas y se definirán más adelante.

In [2]:
# Definimos una lista de archivos ya procesados, de manera que no se carguen todos en cada sesión
archivos_procesados <- readLines('./datos/lista_procesados.txt')

In [3]:
# Agregamos además, functiones que nos permitan actualizar la lista de archivos procesados

guardar_lista <- function(ruta){
    writeLines(archivos_procesados, ruta)
}


actualizar_lista <- function(archivos, reemplazar = FALSE, guardar = FALSE){

    if (reemplazar) {
        archivos_procesados <- archivos
    }
    else {
        archivos_procesados <- c(archivos_procesados, archivos)
        archivos_procesados <- unique(archivos_procesados)
    }

    if (guardar) {
        writeLines(archivos_procesados, './datos/lista_procesados.txt')
    }

}

actualizar_lista_indicar_procesados <- function(archivos_procesando){

    archivos_procesados_nuevos <- c()
    
    for (archivo in archivos_procesando) {

        # Solicitar confirmación
        respuesta <- readline(paste0("¿Desea indicar el archivo [", archivo, "] como procesado? (s/n)"))

        if (tolower(respuesta) == 's') {
            archivos_procesados_nuevos <- c(archivos_procesados_nuevos, archivo)
        }
    }

    respuesta <- readline("Lista actualizada. ¿Desea guardar la lista? (s/n)")

    guardar <- FALSE

    if (tolower(respuesta) == 's') {
        guardar <- TRUE
    }
    
    actualizar_lista(archivos_procesados_nuevos, reemplazar = FALSE, guardar = guardar)
}


Para optimizar el proceso de lectura de archivos, vamos a definir funciones que:

* Descubran los archivos .csv dada una serie de carpetas base.
* Extraigan el nombre de los mismos para renombrarlos de ser necesario.
* Lean los archivos .csv y los conviertan en un dataframe de R.

In [4]:
# Definimos una función que nos permita conocer las rutas de los archivos a procesar, dada una lista de carpetas base
obtener_rutas <- function(carpetas) {

    rutas <- c()

    for (carpeta in carpetas) {

        rutas <- c(
            rutas, # Rutas anteriores
            list.files(carpeta, pattern = "*.csv", full.names = TRUE) # Rutas de los archivos descubiertos
        )
    }

    return(rutas)
}

# Definimos una función que nos de el nombre del archivo a partir de su ruta
obtener_nombre_archivo <- function(ruta) {

    # Separamos la ruta por las diagonales
    ruta_separada <- strsplit(ruta, "/", fixed = TRUE)[[1]]

    # Obtenemos el último elemento de la ruta
    nombre_archivo <- ruta_separada[length(ruta_separada)]

    return(nombre_archivo)
}

Ahora, es momento de realizar una estrategia de lectura de archivos. Debido a que el volumen de tweets suministrados es grande, se optará por leer los archivos uno a uno.

A continuación, únicamente relacionaremos el la ruta del archivo con su nombre, para leerlo en el momento que sea necesario y renombrar el archivo de salida de forma correspondiente.

In [5]:
obtener_archivos_por_leer <- function(carpetas, archivos_procesados, verbose){

    # Obtenemos las rutas de los archivos a procesar
    rutas <- obtener_rutas(carpetas)

    # Se juntan las rutas con los nombres de los archivos

    archivos_sin_leer <- list()

    for (ruta in rutas){

        nombre_archivo <- obtener_nombre_archivo(ruta)

        if (!(nombre_archivo %in% archivos_procesados)) {
            
            # Agregamos el nombre del archivo y su ruta a la lista
            archivos_sin_leer <- append(
                archivos_sin_leer,
                list( # El método append toma las dos listas y las junta en una sola
                    list( # Por eso, la lista contiene otra lista, que es una tupla
                        nombre = nombre_archivo,
                        ruta = ruta
                    )
                )
            )
            
        }

    }

    if (verbose) { # Si se solicita, imprimimos los archivos encontrados

        if (length(archivos_sin_leer) == 0){
            print('No se encontraron archivos nuevos.')
        } else {
            print('Se encontraron los siguientes archivos:')
        }
        
        for (archivo in archivos_sin_leer) {
            print(archivo$nombre)
        }
    }

    return(archivos_sin_leer)
}

# Está función de escritura mantiene el formato en el que originalmente se encuentan los .csv con Tweets
escribir_dataframe <- function(dataframe, ruta_archivo) {
    write.csv(
        dataframe,
        file = ruta_archivo,
        row.names = FALSE,
        col.names = TRUE,
        quote = TRUE
    )
}

Ahora, empleamos las funciones para obtener los archivos que leeremos.

In [6]:
# Leeremos los archivos .csv de las carpetas especificadas
# Hecho de esta manera, emplearemos las rutas para procesar los archivos
carpetas_a_explorar <- c(
    './datos/twitter/'
)

rutas_archivos <- obtener_archivos_por_leer(carpetas_a_explorar, archivos_procesados, verbose = TRUE)

[1] "Se encontraron los siguientes archivos:"
[1] "necesito_ayuda_complete.csv"
[1] "suicida_complete.csv"


In [7]:
# Guardamos los archivos que se van a procesar en una lista

archivos_de_la_sesion <- c()

for (archivo in rutas_archivos) {
    archivos_de_la_sesion <- c(archivos_de_la_sesion, archivo$nombre)
}

Después, reduciremos el tamaño de los conjuntos de datos, y mantendremos sólo aquellos usuarios con N publicaciones o más.

Esto nos permitirá hacer una revisión manual después, para detectar posibles usuarios que no deberían ser considerados en la construcción de los conjuntos de datos limpios.

In [7]:
# Obtenemos del dataframe las filas que tengan usuarios que aparezcan más de N veces

obtener_publicaciones <- function(dataframe, pub_minimas) {

    dataframe <- dataframe %>% 
        group_by(User) %>% 
        filter(n() >= pub_minimas)

    return(dataframe)
}

reducir_archivos <- function(archivos, pub_minimas, excepciones_archivos_a_reducir, ruta_guardado, verbose, sufijo) {

    
    for (ruta_nombre in archivos){

        ruta <- ruta_nombre$ruta
        nombre <- ruta_nombre$nombre

        if (!(nombre %in% excepciones_archivos_a_reducir)){

            # Leemos el archivo

            if (verbose){
                print(paste('Leyendo el archivo', nombre))
            }

            dataframe <- read.csv(file = ruta)

            filas_iniciales <- nrow(dataframe)

            # Obtenemos las publicaciones de los usuarios que aparezcan más de N veces

            if (verbose){
                print(paste('Filtrando usuarios con menos de', pub_minimas, 'publicaciones'))
            }

            dataframe <- obtener_publicaciones(dataframe, numero_min_publicaciones)

            filas_finales <- nrow(dataframe)

            # Creamos la nueva ruta. Le quitamos el .csv al nombre del archivo
            nueva_ruta <- paste0(ruta_guardado, substr(nombre, 1, nchar(nombre) - 4), sufijo)

            # Escribimos el archivo
            escribir_dataframe(dataframe, nueva_ruta)

            if (verbose){
                print(
                    paste('El archivo',
                            nombre,
                            '(# ',
                            filas_iniciales ,
                            'líneas) se redujo ( ahora',
                            obtener_nombre_archivo(nueva_ruta),
                            ', #',
                            filas_finales,
                            ' líneas).'
                    )
                )
            }

        }
        else if (verbose){
            print(paste('El archivo', nombre, 'no se redujo.'))
        }

    }
}

Ahora que ya definimos este paso del pipeline, es momento de ejecutarlo.

Toma en cuenta que necesitas crear el directorio en el que vas a guardar los archivos de salida.

In [27]:
# Si lo deseamos, podemos indicar archivos que no queremos pasar por el proceso de reducción
excepciones_archivos_a_reducir <- c(
    
)

numero_min_publicaciones <- 10

reducir_archivos(
    rutas_archivos,
    numero_min_publicaciones, 
    excepciones_archivos_a_reducir,
    ruta_guardado = './datos_limpios/datos_por_revisar/twitter/',
    verbose = TRUE,
    sufijo = '_reducido.csv'
)

[1] "Leyendo el archivo necesito_ayuda_complete.csv"
[1] "Filtrando usuarios con menos de 10 publicaciones"


Warning message in write.csv(dataframe, file = ruta_archivo, row.names = FALSE, :
"attempt to set 'col.names' ignored"


[1] "El archivo necesito_ayuda_complete.csv (#  4198034 líneas) se redujo ( ahora necesito_ayuda_complete_reducido.csv , # 748962  líneas)."
[1] "Leyendo el archivo suicida_complete.csv"
[1] "Filtrando usuarios con menos de 10 publicaciones"


Warning message in write.csv(dataframe, file = ruta_archivo, row.names = FALSE, :
"attempt to set 'col.names' ignored"


[1] "El archivo suicida_complete.csv (#  2816694 líneas) se redujo ( ahora suicida_complete_reducido.csv , # 1092681  líneas)."


In [21]:
# Guardar el progreso en la sesión
actualizar_lista_indicar_procesados(archivos_de_la_sesion)

Debido a que existen muchas palabras de poca relevancia y múltiples frases (a veces generadas a través de aplicaciones, otras son letras de canciones que usan la palabra con otro significado, y a veces incluso son etiquetas a usuarios populares o de influencia política), las celdas siguientes atienden necesidades específicas de la limpieza de cada conjunto de datos.

No obstante, por practicidad, se declaran previamente las funciones que se emplearán para limpiar los conjuntos de datos, con el fin de declararlas en la sesión más accesiblemente.

In [8]:
# Se juntan los vectores que contienen los nombres de los usuarios, para ser unificados en una sola lista negra
# Se le suministrarán varios vectores de usuarios, y se devolverá un vector con los usuarios únicos

juntar_usuarios <- function(listas_usuarios) {

    usuarios <- c()

    for (lista in listas_usuarios) {
        usuarios <- c(usuarios, lista)
    }

    usuarios <- unique(usuarios)

    return(usuarios)
}

In [11]:
# Función que obtiene los tweets de un usuario y determina si todos contienen una serie de textos claves

tweetsSonRepetitivos <- function(dataset, usuario, textosClave) {
    # Obtenemos los tweets del usuario
    tweets <- dataset %>%
        filter(User == usuario) %>%
        pull(Content) # pull() devuelve un vector con los valores de la columna
    
    # Revisamos si todos los tweets contienen alguno de los textos clave
    for (texto in textosClave){
        todosContienen <- all(grepl(texto, tweets, ignore.case = TRUE))

        # Nos conformamos con saber que uno de los textos que deseamos excluir está en los tweets,
        # pues es suficiente para determinar que el usuario repite contenido
        if (todosContienen) {
            return (TRUE)
        }
    }
    
    return (todosContienen)
}

# Función que toma un dataset y una serie de textos clave, y devuelve los usuarios que tienen todos
# sus tweets con al menos uno de esos textos clave

obtenerUsuariosRepetitivos <- function(dataset, textosClave) {
    # Obtenemos los usuarios
    usuarios <- dataset %>%
        distinct(User) %>%
        pull(User)
        
    # Obtenemos los usuarios que tienen todos sus tweets con alguno de sus textos clave en este vector
    usuarios_repetitivos <- c()
    
    for (usuario in usuarios) {

        if (tweetsSonRepetitivos(dataset, usuario, textosClave)) {
            usuarios_repetitivos <- c(usuarios_repetitivos, usuario)
            break
        }
    }
    
    return (unique(usuarios_repetitivos))
}

In [16]:
raiz_verbo_aparece <- function(texto, raices_a_comparar, lenguaje) {

    palabras <- unlist(strsplit(texto, " "))
    raices_palabras <- wordStem(palabras, language = lenguaje)
    
    for (raiz_palabra in raices_palabras) {
        
        for (raiz_verbo in raices_a_comparar) {
            
            if ( grepl(raiz_verbo, raiz_palabra, ignore.case = TRUE) ) {
                return(TRUE)
            }
        }

    }

    return(FALSE)

}

# Usando el paquete SnowballC, se buscarán los tweets que contengan alguna conjugación de los verbos indicados

excluir_tweets_segun_verbos <- function(dataframe, palabras, language) {

    # Se obtienen las raíces de las palabras
    raices_palabras <- wordStem(palabras, language = language)

    # Se obtienen los tweets que no contengan ninguna de las raíces de las palabras (i.e., el verbo no aparece
    # en ninguna conjugación)
    tweets_sin_palabras <- dataframe %>%
        mutate(
            contiene_raices_palabras = sapply(Content, raiz_verbo_aparece, raices_palabras, language)
        ) %>%
        filter(contiene_raices_palabras == FALSE) %>%
        select(-contiene_raices_palabras) # Se elimina la columna auxiliar

    return(tweets_sin_palabras)
}

excluir_tweets_segun_expresiones <- function(dataframe, expresiones) {

    # Se obtienen los tweets que no contengan ninguna de las expresiones
    tweets_sin_palabras <- dataframe %>%
        mutate(
            contiene_expresiones = as.vector(sapply(Content, comparar_tweet_con_palabras, expresiones))
        ) %>%
        filter(contiene_expresiones == FALSE) %>%
        select(-contiene_expresiones) # Se elimina la columna auxiliar

    return(tweets_sin_palabras)
}

comparar_tweet_con_palabras <- function(tweet, palabras) {

    # Se revisa si alguna de las palabras despreciables está en el tweet

    for (palabra in palabras) {

        if (grepl(palabra, tweet, ignore.case = TRUE)) {
            return(TRUE)
        }
    }

    return(FALSE)

}

# Función que devuelve los usuarios que usan al menos una de las palabras despreciables indicadas en sus tweets

obtener_usuarios_segun_palabras <- function(dataframe, palabras) {

    # Se obtienen los usuarios que usan al menos una de las palabras despreciables indicadas en sus tweets
    usuarios_con_palabras <- c()

    # Se recorren todos los tweets, y se revisa si alguno contiene alguna de las palabras despreciables,
    # para determinar si el usuario que lo escribió debe ser incluido en la lista negra

    usuarios_con_palabras <- dataframe %>%
        mutate(
            contiene_palabras = as.vector(sapply(Content, comparar_tweet_con_palabras, palabras))
        ) %>%
        filter(contiene_palabras == TRUE) %>%
        distinct(User)

    return (usuarios_con_palabras$User)

}

In [10]:
cargar_lista_negra <- function(ruta = './datos_limpios/lista_negra_revisada.txt') {

    lista_negra <- readLines(ruta)

    return(lista_negra)
}

escribir_lista_negra <- function(lista_negra, ruta = './datos_limpios/lista_negra_revisada.txt', retirar_duplicados = TRUE) {

    if(retirar_duplicados) {
        lista_negra <- unique(lista_negra)
    }
    
    writeLines(lista_negra, ruta)

}

ruta_lista_negra <- './datos_limpios/lista_negra_revisada.csv'

In [11]:
retirar_tweets_usuarios <- function(dataframe, usuarios) {
    return (
        dataframe %>%
            filter(!User %in% usuarios)
    )
}

### Datasets: `ahorcarme_complete.csv` y `colgarme_complete.csv`

In [89]:
# Obtenemos los datos del archivo 'ahorcarme_complete.csv'
publicaciones_ahorcarme <- obtener_dataframe('ahorcarme_complete.csv', datos)
publicaciones_colgarme <- obtener_dataframe('colgarme_complete.csv', datos)

In [4]:
# Filtramos los usuarios que tengan más de 5 publicaciones
publicaciones_ahorcarme_reducidas <- obtener_publicaciones(publicaciones_ahorcarme, 5)
publicaciones_colgarme_reducidas <- obtener_publicaciones(publicaciones_colgarme, 5)

In [6]:
# Creamos la lista de los usuarios que aparecen más de 5 veces en las publicaciones

usuarios_ahorcarme <- publicaciones_ahorcarme_reducidas$User
usuarios_colgarme <- publicaciones_colgarme_reducidas$User

lista_negra <- juntar_usuarios(
    c(
        usuarios_ahorcarme,
        usuarios_colgarme
    )
)

In [9]:
head(lista_negra)

[1] "soiiviic"      "gatoacostadou" "Lucasnun_"     "cigarettommy" 
[5] "lwtironic"     "CyanideMoon"

Ahora, podemos revisar manualmente los tweets de los usuarios que colocamos en la lista negra bajo este criterio, para determinar si realmente sus publicaciones son despreciables para el análisis.

In [7]:
# Obtenemos los tweets de estos usuarios, para revisarlos manualmente

# Ahorcarme
tweets_lisneg_ahorcarme <- publicaciones_ahorcarme_reducidas %>%
    filter(User %in% lista_negra)

# Colgarme
tweets_lisneg_colgarme <- publicaciones_colgarme_reducidas %>%
    filter(User %in% lista_negra)

In [11]:
# Guardamos los tweets en un archivo

# Ahorcarme
escribir_dataframe(tweets_lisneg_ahorcarme, "./datos_limpios/datos_por_revisar/tweets_lisneg_ahorcarme.csv")
# Colgarme
escribir_dataframe(tweets_lisneg_colgarme, "./datos_limpios/datos_por_revisar/tweets_lisneg_colgarme.csv")

Luego de revisar los tweets, encontré un fragmento que hace alusión a una connotación sexual de la palabra concurrente. Por lo tanto, aquellos usuarios cuyas publicaciones que contienen la palabra "ahorcarme" serán añadidos a la lista negra.

Es importante considerar que, si bien emplean la palabra bajo un concepto sexual, puede no ser la única forma en la que la usan; de hecho, logré encontrar varios perfiles en los que hacen un uso sexual y violento del verbo. De tal manera, de este grupo de usuarios que tergiversan de esta forma la acción, únicamente se seleccionarán aquellos que emplean la palabra exclusivamente en un contexto sexual, es decir, en todos los tweets que les fueron recolectados que les pertenecen.

In [14]:
# Fragmento
frags_frase = c(
    "ahorcame por dios ahorcame hasta el punto de que tus manos"
)

# Localizar los tweets que contengan la frase, y de ellos tomar los usuarios, para agregarlos a los que ya
# estaban listados, siempre y cuando todos los tweets del usuario contengan la frase

usuarios_frags_ahorcarme <- obtenerUsuariosRepetitivos(publicaciones_ahorcarme, frags_frase)

Otro son fragmentos de una canción, en los tweets con la palabra clave "colgarme"

Los fragmentos son los siguientes:

<blockquote>
    <ul>
        <li> Niña, dame una pestaña de tus ojos para colgarme de amor por ti </li>
        <li> Colgarme de cualquiera que le gusta trasnochar </li>
    </ul>
</blockquote>

In [16]:
# Fragmentos
frags_frase = c(
    "Niña, dame una pestaña de tus ojos para colgarme de amor por ti",
    "Colgarme de cualquiera que le gusta trasnochar"
)
# Localizar los tweets que contengan la frase, y de ellos tomar los usuarios, para agregarlos a los que ya
# estaban listados.

usuarios_frags_colgarme <- obtenerUsuariosRepetitivos(publicaciones_colgarme, frags_frase)

Igualmente, se removerá cualquier uso de la llamada en el contexto de una conversación telefónica.

Por su parte, otros usos de la palabra que se refieran a una acción como "depender de" o "aprovecharse de" podrían requerir de otro tipo de servicios para ser procesados.

En [este notebook](./filtrado_publicaciones_por_contexto.ipynb) estaré trabajando en otros usos de la palabra a través de los servicios que ofrece Microsoft Azure. Indagaré en el uso de los servicios cognitivos o las integraciones con OpenAI para proveer un filtro de las publicaciones con un procesamiento del lenguaje más intuitivo y preciso.

In [9]:
# No quitamos al usuario completamente porque el uso de la palabra "colgar" en una llamada no le excluye de usarla con
# intenciones autolesivas, por lo que nos limitamos a excluir esas publicaciones

palabras_contexto_telefonico <- c(
    "llamar",
    "marcar",
    "responder",
    "contestar"
)

# Demostrando el uso del paquete tm
raices_palabras_contexto_telefonico <- wordStem(palabras_contexto_telefonico, language = "spanish")
cat(raices_palabras_contexto_telefonico)

[1] "llam"    "marc"    "respond" "contest"


In [28]:
# Eliminamos los tweets que contengan alguna conjugación de los verbos que indican que se habla de una llamada

tweets_colgarme_limpios <- excluir_tweets_segun_verbos(
    publicaciones_colgarme,
    palabras_contexto_telefonico,
    "spanish"
)

In [29]:
# Escribir tweets para no depender de la sesión de R
escribir_dataframe(tweets_colgarme_limpios, "./datos_limpios/datos_por_revisar/tweets_colgarme_limpios.csv")

In [36]:
# Escribir los mismos tweets, pero extrayendo aquellos cuyos usuarios aparecen con frecuencia

escribir_dataframe(
    obtener_publicaciones(tweets_colgarme_limpios, 5),
    "./datos_limpios/datos_por_revisar/tweets_colgarme_limpios_5_ocurrencias.csv"
)

In [31]:
# Agregar los usuarios a la lista de usuarios

lista_negra <- juntar_usuarios(
    c(
        lista_negra,
        usuarios_frags_ahorcarme,
        usuarios_frags_colgarme
    )
)

In [30]:
# Escribimos las listas de usuarios para no depender de la sesión de R
writeLines(usuarios_frags_ahorcarme, "./datos_limpios/datos_por_revisar/usuarios_frag_ahorcarme.txt")
writeLines(usuarios_frags_colgarme, "./datos_limpios/datos_por_revisar/usuarios_frag_colgarme.txt")

In [32]:
# Escribir la lista negra en un archivo
writeLines(
    lista_negra,
    file = "./datos_limpios/datos_por_revisar/lista_negra.txt",
)

A partir de aquí, una vez que hallamos revisado la lista negra de usuarios, vamos a limpiar los conjuntos de datos para obtener únicamente la información útil.

In [33]:
# Excluir a estos usuarios de los datasets, pero a partir de la lista negra de usuarios ya revisada

# Leer lista
lista_negra_revisada <- readLines("./datos_limpios/lista_negra_revisada.txt", encoding = "UTF-8")

In [35]:
# Reescribir lista negra, para obtener sólo valores únicos
lista_negra_revisada <- unique(lista_negra_revisada) %>% sort()
writeLines(lista_negra_revisada, "./datos_limpios/lista_negra_revisada.txt")

In [ ]:
# Ahorcarme
publicaciones_ahorcarme <- retirar_tweets_usuarios(publicaciones_ahorcarme, lista_negra_revisada)

# Colgarme
publicaciones_colgarme <- retirar_tweets_usuarios(tweets_colgarme_limpios, lista_negra_revisada)

In [ ]:
# Guardamos los tweets en un archivo

# Ahorcarme
escribir_dataframe(publicaciones_ahorcarme, "./datos_limpios/ahorcarme_filtered.csv")

# Colgarme
escribir_dataframe(publicaciones_colgarme, "./datos_limpios/colgarme_filtered.csv")

### Dataset `necesito_ayuda_complete.csv`

En este dataset encontramos las siguientes stopwords:

- Estoy jugando #VenezuelaQuiz y necesito ayuda con esta imagen ¿La reconoces?
- ¡Necesito ayuda con mis árboles!
- Entra aquí
- Entra en
- _Usuarios con “1D” en su nombre (fans de One Direction solicitando ayuda para conseguir boletos)_
- One Direction
- sorteo
- concurso
- Sigan a
- Síganme
- rt
- retweet
- MG
- MG AL COMENTARIO
- bonus

Las usaremos para obtener a usuarios que utilicen esas frases en sus publicaciones, y los añadiremos a la lista negra.

In [12]:
# Se lee el dataset

datos_necesito_ayuda <- read.csv(file = "./datos/twitter/necesito_ayuda_complete.csv")

In [13]:
stopwords_necesito_ayuda <- c(
    "Estoy jugando #VenezuelaQuiz y necesito ayuda con esta imagen ¿La reconoces?",
    "¡Necesito ayuda con mis árboles!",
    "Entra aquí",
    "Entra en",
    "1D",
    "One Direction",
    "sorteo",
    "concurso",
    "Sigan a",
    "Siganme",
    "Síganme",
    "rt",
    "retweet",
    "retuit",
    "MG",
    "MG AL COMENTARIO",
    "bonus"
)

In [30]:
usuarios_ls_negra_necesito_ayuda <- obtener_usuarios_segun_palabras(
    dataframe = datos_necesito_ayuda,
    palabras = stopwords_necesito_ayuda
)

datos_necesito_ayuda_limpios <- retirar_tweets_usuarios(
    datos_necesito_ayuda,
    usuarios_ls_negra_necesito_ayuda
)

In [38]:
# Agregamos los usuarios a la lista negra
lista_negra_revisada <- cargar_lista_negra()
lista_negra_revisada <- juntar_usuarios(
    c(
        lista_negra_revisada,
        usuarios_ls_negra_necesito_ayuda
    )
)

escribir_lista_negra(lista_negra_revisada)

Warning message in readLines(ruta):
"incomplete final line found on './datos_limpios/lista_negra_revisada.txt'"


Además, retiraremos tweets que contengan enlaces

In [40]:
datos_necesito_ayuda_limpios <- excluir_tweets_segun_expresiones(
    dataframe = datos_necesito_ayuda_limpios,
    expresiones = c(
        "https://",
        "http://"
    )
)

In [41]:
escribir_dataframe(
    datos_necesito_ayuda_limpios,
    "./datos_limpios/twitter/necesito_ayuda_cleaned.csv"
)

Warning message in write.csv(dataframe, file = ruta_archivo, row.names = FALSE, :
"attempt to set 'col.names' ignored"


Encontramos, además, propaganda política en los tweets de los usuarios. A menudo, etiquetan al (al año de 2023) presidente de Venezuela, Nicolás Maduro, en sus publicaciones.

No obstante, si revisamos estas cuentas podremos percatarnos de que en sus publicaciones existen peticiones por recursos, pues son personas en situación de calle, desempleo, o que no tienen acceso a servicios básicos. Por ello, se tomarán estos usuarios y se colocaran en una "lista gris", para que puedan ser examinados después por especialistas y determinar si su situación les puede suscitar conductas depresivas.

### Dataset `perdon_por_todo_complete.csv`

En este dataset encontramos estas frases que podrían ser de interés excluir:

In [ ]:
expresiones_perdon_por_todo <- c(
    "Perdón por ser fiel, entregar todo de mí, quererte como a nadie",
    "Gracias por enseñarme lo que debo mejorar",
    "al querer ser perfecto se equivoca todo el mundo",
    "Ya no queda valor para mirarnos de nuevo y pedirnos",
    "Gracias por todo y perdón por ser tan básico",
    "Corazón, perdón por todo el daño"
)

datos_perdon_por_todo <- read.csv(file = "./datos/twitter/perdon_por_todo_complete.csv")

In [ ]:
usuarios_ls_negra_perdon <- obtenerUsuariosRepetitivos(
    dataset = datos_perdon_por_todo,
    textosClave = expresiones_perdon_por_todo
)

In [ ]:
# Agregamos los usuarios a la lista negra
lista_negra_revisada <- cargar_lista_negra()
lista_negra_revisada <- juntar_usuarios(
    c(
        lista_negra_revisada,
        usuarios_ls_negra_necesito_ayuda
    )
)

escribir_lista_negra(lista_negra_revisada)

In [ ]:
# Retiramos los usuarios del dataset y los guardamos en un archivo
datos_perdon_por_todo_limpios <- retirar_tweets_usuarios(
    datos_perdon_por_todo,
    usuarios_ls_negra_perdon
)

# Retiramos los tweets con URLs

datos_perdon_por_todo_limpios <- excluir_tweets_segun_expresiones(
    dataframe = datos_perdon_por_todo_limpios,
    expresiones = c(
        "https://",
        "http://",
    )
)

escribir_dataframe(
    datos_perdon_por_todo_limpios,
    "./datos_limpios/perdon_por_todo_cleaned.csv"
)